# Exploratory Data Analysis

## Audience/Stakeholders

- Clearly identify who your are writing your final report for

This final report on the performance of UVA men's basketball is intended for the Coaching Staff, as well as the Athletic Department. These stakeholders are involved in studying and making decisions about player developments, training loads, game preparations, and long-term performance strategies.

## Problem Statement

- Create a concise and compelling problem or question that guides your analysis.
  - Ex. "The men's basketball team had a worse performance this year. Did preparation around games change? How did playerload, jumps, high accelerations, and change of direction compare before games over the current season and the previous season?"
  
In today’s college basketball climate where players often jump from school to school, many programs have struggled to adapt. Many programs, like UVA, have historically relied on player development with the convention of players staying at a single school throughout their college career. Given the shifting landscape, we want to analyze whether or not a single season is enough for a player to truly develop and progress. We will look at the Catapult data of several players over the course of a season, tracking how their PlayerLoad/min and jump/acceleration band distributions evolve. We want to measure individual growth with the aim of driving scouting decisions. Should coaches scout players with the anticipation that significant player development can happen over the course of just one season or should they recruit more talented players with less room for growth under the assumption that one season isn’t enough for a player to mature?

## Important Variables

- List which ones are important for your analysis and why.

## Merging and Cleaning the Dataset

- Clean the data: Remove duplicates, handle missing values, correct data types

- Your final dataset should include only variables relevant to your problem

In [53]:
# Imports
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import warnings

# Loading in data from season 1 and 2
s1 = pd.read_csv("../data/catapult season 1.csv")
s2 = pd.read_csv("../data/catapult season 2.csv")

# Adding column with season to each dataset
s1["Season"] = 1
s2["Season"] = 2

# Combining the two seasons of data into a single data frame
data = pd.concat([s1, s2])
# Ignoring the UserWarning raised by the above line. The warning raised tells me that not setting a specific format or handing errors could be problematic, but further
# examination of the Date column showed that is not, so I am hiding the warning from printing
warnings.simplefilter(action = "ignore", category = UserWarning)

# Converting date column to datetime
data["Date"] = pd.to_datetime(data["Date"])

data.head(1)

,Date,About,Position,Period Number,Period,Total Acceleration Efforts,Total Player Load,Player Load Per Minute,IMA Accel Low,IMA Decel Low,...,Session Total Jump,Session Jumps Per Minute,Total CoD Left,Total CoD Right,Total High IMA,Total IMA,IMA/Min,event-uuid,group-uuid,Season
0,2023-03-14,Athlete I,Guard,1,1. Pre Practice,0,87.437,4.1,3,17,...,95.0,1.05,269.0,306.0,89.0,899.0,NaN,c4e1f0fe-b87a-42ca-8f41-b5b0e4cdfab3,c4e1f0fe-b87a-42ca-8f41-b5b0e4cdfab3,1


In [54]:
columns_of_interest = ["Date", "About", "Period", "Player Load Per Minute", "IMA Accel Low", "IMA Accel High", "IMA Jump Count Low Band", "IMA Jump Count Med Band", "IMA Jump Count High Band", "Season"]

data = data[columns_of_interest]

games = data[data["Period"].str.contains("Period") == True]
games = games[games["Period"].str.contains("Play") == False]
games = games[games["Period"].str.contains("Auto") == False]
games.loc[games["Period"].str.contains("Period 1"), "Period"] = "Game Half 1"
games.loc[games["Period"].str.contains("Period 2"), "Period"] = "Game Half 2"
games.loc[games["Period"].str.contains("Period 3"), "Period"] = "Overtime"
games.head()

,Date,About,Period,Player Load Per Minute,IMA Accel Low,IMA Accel High,IMA Jump Count Low Band,IMA Jump Count Med Band,IMA Jump Count High Band,Season
273,2023-03-11,Athlete H,Game Half 1,12.8,25,7,12,4,6,1
274,2023-03-11,Athlete H,Game Half 2,10.2,26,16,16,5,5,1
276,2023-03-11,Athlete J,Game Half 1,10.7,54,20,19,12,6,1
277,2023-03-11,Athlete J,Game Half 2,10.9,37,6,21,6,6,1
279,2023-03-11,Athlete B,Game Half 1,12.6,5,1,6,1,0,1


In [55]:
scrims = data[data["Period"].str.contains("Scrim")]
scrims = scrims[scrims["Period"].str.contains("Full")]
scrims = scrims[scrims["Period"].str.contains("Seven") == False]
scrims = scrims[scrims["Period"].str.contains("Basket") == False]
scrims["Period"] = "Scrimmage"
scrims.head()

,Date,About,Period,Player Load Per Minute,IMA Accel Low,IMA Accel High,IMA Jump Count Low Band,IMA Jump Count Med Band,IMA Jump Count High Band,Season
11313,2022-11-15,Athlete D,Scrimmage,11.8,5,0,3,2,0,1
11315,2022-11-15,Athlete D,Scrimmage,0.5,0,0,0,0,0,1
11331,2022-11-15,Athlete I,Scrimmage,11.5,6,2,1,1,3,1
11333,2022-11-15,Athlete I,Scrimmage,10.2,8,1,4,1,1,1
11349,2022-11-15,Athlete L,Scrimmage,12.9,10,4,1,1,0,1


In [59]:
data = pd.concat([games, scrims], axis = 0)
data.head()

,Date,About,Period,Player Load Per Minute,IMA Accel Low,IMA Accel High,IMA Jump Count Low Band,IMA Jump Count Med Band,IMA Jump Count High Band,Season
273,2023-03-11,Athlete H,Game Half 1,12.8,25,7,12,4,6,1
274,2023-03-11,Athlete H,Game Half 2,10.2,26,16,16,5,5,1
276,2023-03-11,Athlete J,Game Half 1,10.7,54,20,19,12,6,1
277,2023-03-11,Athlete J,Game Half 2,10.9,37,6,21,6,6,1
279,2023-03-11,Athlete B,Game Half 1,12.6,5,1,6,1,0,1


## Descriptive Statistics & Distributions

- Provide Summaries of important variables

- Use visualizations to explore distributions

## Examine Correlations (If Relevant)

- Interpret Findings: What variables appear related?

## Explore Relationship (If Relevant)

- Dig into potential causal or descriptive relationships

- Use visualizations and statistical summaries